In [53]:
import pandas as pd
from prophet import Prophet
from datetime import datetime, timedelta
import duckdb


In [54]:
# Conecta abrindo / criando um arquivo
conn2 = duckdb.connect('./dados/hackathon_2025_2.duckdb')

In [55]:
# Gravação no formato Parquet
df_transacao = conn2.execute("""
    SELECT
    *
    FROM
    read_parquet('./dados/hackathon_2025_templates/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')
""").fetchdf()
df_transacao.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


In [56]:
df_transacao.shape

(6560698, 11)

In [ ]:
# 1. Carregar os dados

# Copia apenas 1000 registros e já renomeia as colunas
novo_df = df_transacao.loc[:99999,['transaction_date', 'internal_store_id', 'internal_product_id', 'quantity', 'gross_value']].rename(
    columns={
        'transaction_date': 'Data',
        'internal_store_id': 'PDV',
        'internal_product_id': 'Produto',
        'quantity': 'Quantidade',
        'gross_value': 'Faturamento'
    }
).copy()


In [58]:
novo_df

,Data,PDV,Produto,Quantidade,Faturamento
0,2022-07-13,7384367747233276219,328903483604537190,1.0,38.125000
1,2022-03-21,3536908514005606262,5418855670645487653,6.0,107.250000
2,2022-09-06,3138231730993449825,1087005562675741887,3.0,56.625000
3,2022-09-11,3681167389484217654,1401422983880045188,129.0,1037.160023
4,2022-02-18,7762413312337359369,6614994347738381720,1.0,26.230000
...,...,...,...,...,...
99995,2022-09-27,965108893528640767,8174625658473329985,3.0,101.699997
99996,2022-11-25,4571133542649639528,2014744907596959599,1.0,26.600000
99997,2022-06-23,6813271863163885540,4046308726663975437,2.0,56.599998
99998,2022-01-05,1647189459014103120,1085359296765146304,3.0,102.525002


In [59]:
# 2. Agregar por semana, PDV e Produto
novo_df['Semana'] = novo_df['Data'].dt.to_period('W').apply(lambda r: r.start_time)

In [60]:
df_transacao_agg = novo_df.groupby(['Semana', 'PDV', 'Produto'])['Quantidade'].sum().reset_index()


In [61]:
df_transacao_agg

,Semana,PDV,Produto,Quantidade
0,2021-12-27,1073673009638227380,819652267660746406,2.0
1,2021-12-27,1244392535353757027,7170592116007465797,1.0
2,2021-12-27,1515114354837721111,3262679882836704514,10.0
3,2021-12-27,1629395651634182934,2137299631507474345,1.0
4,2021-12-27,2769047624841228049,1288112997726060694,1.0
...,...,...,...,...
99930,2022-12-26,987901627121453256,4872583472354874832,1.0
99931,2022-12-26,989430645247912212,5418855670645487653,17.0
99932,2022-12-26,99018150325538996,4098058333001424920,1.0
99933,2022-12-26,990571711439878263,637448570435263100,2.0


In [62]:
# 3. Gerar previsões por PDV/SKU
resultados = []

i = 0
# 4. Loop por cada combinação única
for (pdv, produto), grupo in df_transacao_agg.groupby(['PDV', 'Produto']):
    # Preparar dados para Prophet
    ts = grupo[['Semana', 'Quantidade']].rename(columns={'Semana': 'ds', 'Quantidade': 'y'})

    #print(f"Treinando modelo para PDV: {pdv}, Produto: {produto} - Série temporal com {len(ts)} pontos")

    # Verificar se há dados suficientes
    if len(ts) < 2:
        print(f"⚠️ Ignorando PDV: {pdv}, Produto: {produto} - Série temporal muito curta com {len(ts)} pontos")
        continue  # Ignora séries muito curtas

    # Criar e treinar o modelo
    modelo = Prophet(weekly_seasonality=True, daily_seasonality=False)
    modelo.fit(ts)

    # Criar datas futuras (5 semanas de janeiro/2023)
    last_date = ts['ds'].max()
    # Garante que teremos datas até pelo menos o final de janeiro/2023
    weeks_needed = ((datetime(2023, 2, 6) - last_date).days // 7) + 5
    future = modelo.make_future_dataframe(periods=max(weeks_needed, 5), freq='W')
    future = future[future['ds'] >= datetime(2023, 1, 1)]

    # Verifica se o dataframe future não está vazio antes de prever
    if future.empty:
        continue

    # Fazer previsão
    forecast = modelo.predict(future)

    # Extrair resultados
    previsoes = forecast[['ds', 'yhat']].copy()
    previsoes['PDV'] = pdv
    previsoes['Produto'] = produto
    previsoes['yhat'] = previsoes['yhat'].apply(lambda x: max(0, round(x)))  # Evita valores negativos

    resultados.append(previsoes)


⚠️ Ignorando PDV: 1000285336744471896, Produto: 4040509988492387426 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 1009179103632945474 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 1029370090212151375 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 145852603040678098 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 1527082310248040324 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 1625722803643187564 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 18540138743288609 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 2393101641949628355 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1001371918471115422, Produto: 2423393825246291901 - Série temporal muito curta com 1 pontos
⚠️ Ignorando 

21:01:34 - cmdstanpy - INFO - Chain [1] start processing
21:01:34 - cmdstanpy - INFO - Chain [1] done processing
21:01:34 - cmdstanpy - INFO - Chain [1] start processing


⚠️ Ignorando PDV: 1004005665140505784, Produto: 2120799115839400862 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 3262679882836704514 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 3648010604036016587 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 4040509988492387426 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 429738700876868379 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 4303800370307860749 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 4623814317972718932 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 4853551828912066478 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004005665140505784, Produto: 500478784353013717 - Série temporal muito curta com 1 pontos
⚠️ Ignorando

21:01:34 - cmdstanpy - INFO - Chain [1] done processing


⚠️ Ignorando PDV: 1004779246734143594, Produto: 145852603040678098 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 2021554495441138153 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 2645522295208024004 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 347520348779130794 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 6416577388566874702 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 6520870625209395817 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 6529217055583158447 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1004779246734143594, Produto: 9201926030466239921 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 1005375718888197324, Produto: 1029370090212151375 - Série temporal muito curta com 1 pontos
⚠️ Ignorando

...

⚠️ Ignorando PDV: 2809621131987776420, Produto: 8456670153959774587 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809621131987776420, Produto: 8581872597368912387 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809621131987776420, Produto: 8960695008373210791 - Série temporal muito curta com 1 pontos


21:24:33 - cmdstanpy - INFO - Chain [1] done processing
21:24:33 - cmdstanpy - INFO - Chain [1] start processing
21:24:34 - cmdstanpy - INFO - Chain [1] done processing
21:24:34 - cmdstanpy - INFO - Chain [1] start processing
21:24:34 - cmdstanpy - INFO - Chain [1] done processing


⚠️ Ignorando PDV: 2809712975942573469, Produto: 2258747115919456585 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 3371701102715312626 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 3407816598812292920 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 3435249648591979862 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 4439969214198699755 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 5691135100585682448 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 6542133380128408096 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 6944668232060731777 - Série temporal muito curta com 1 pontos
⚠️ Ignorando PDV: 2809712975942573469, Produto: 78720985238870578 - Série temporal muito curta com 1 pontos
⚠️ Ignorando

21:24:34 - cmdstanpy - INFO - Chain [1] start processing
21:24:34 - cmdstanpy - INFO - Chain [1] done processing
21:24:34 - cmdstanpy - INFO - Chain [1] start processing


KeyboardInterrupt: 

In [ ]:
resultados

[           ds  yhat                  PDV              Produto
 0  2023-01-01     4  3266508836797762409  8625590539951587748
 1  2023-01-08     4  3266508836797762409  8625590539951587748
 2  2023-01-15     4  3266508836797762409  8625590539951587748
 3  2023-01-22     4  3266508836797762409  8625590539951587748
 4  2023-01-29     4  3266508836797762409  8625590539951587748
 5  2023-02-05     4  3266508836797762409  8625590539951587748
 6  2023-02-12     4  3266508836797762409  8625590539951587748
 7  2023-02-19     4  3266508836797762409  8625590539951587748
 8  2023-02-26     3  3266508836797762409  8625590539951587748
 9  2023-03-05     3  3266508836797762409  8625590539951587748
 10 2023-03-12     3  3266508836797762409  8625590539951587748,
            ds  yhat                  PDV              Produto
 0  2023-01-01     2  5260184350830305073  7418598396863486078
 1  2023-01-08     2  5260184350830305073  7418598396863486078
 2  2023-01-15     2  5260184350830305073  74185983968

In [ ]:
# 5. Consolidar tudo
df_transacao_final = pd.concat(resultados)
df_transacao_final = df_transacao_final.rename(columns={'ds': 'Semana', 'yhat': 'Quantidade_Prevista'})

# 6. Salvar resultado
#df_transacao_final.to_csv('previsao_vendas_jan2023.csv', index=False)

print("✅ Previsão gerada com sucesso para janeiro/2023!")
df_transacao_final.head()

✅ Previsão gerada com sucesso para janeiro/2023!


,Semana,Quantidade_Prevista,PDV,Produto
0,2023-01-01,4,3266508836797762409,8625590539951587748
1,2023-01-08,4,3266508836797762409,8625590539951587748
2,2023-01-15,4,3266508836797762409,8625590539951587748
3,2023-01-22,4,3266508836797762409,8625590539951587748
4,2023-01-29,4,3266508836797762409,8625590539951587748


In [ ]:
conn2.close()